# Tool Call LLMs
Some of the more exciting use cases for RC is when you want to cede control of the decision making process to the LLM. RC has a suite of tools that you can use to make this process much simpler. Check out an example below. 

In [ ]:
from typing import Dict, Any, Set, Type
import requestcompletion as rc

In [2]:
# Common params for all terminal_llms being used as tools in this example
critic_tool_params = {
    rc.llm.Parameter("analysis_detail", "string", "The thing you would like to analyze")
}

critic_model = rc.llm.OpenAILLM("gpt-4o")

In [4]:
HarshCritic = rc.library.terminal_llm(
    pretty_name="Harsh Critic",
    system_message=rc.llm.SystemMessage(
        "You are a harsh critic of the world and you should analyze the given statement and provide a harsh critique of it. Be very concise and to the point."
    ),
    model=critic_model,
    tool_details="A tool used to critique a statement harshly.",
    tool_params=critic_tool_params,
)

PositiveCritic = rc.library.terminal_llm(
    pretty_name="Positive Critic",
    system_message=rc.llm.SystemMessage(
        "You are a positive critic of the world and you should analyze the given statement and provide a positive critique of it. Be very concise and to the point."
    ),
    model=critic_model,
    tool_details="A tool used to critique a statement positively.",
    tool_params=critic_tool_params,
)

DeeperMeaningCritic = rc.library.terminal_llm(
    pretty_name="Deeper Meaning Critic",
    system_message=rc.llm.SystemMessage(
        "You are a critic of the world and you should analyze the given statement and provide a critique of it. Be very concise and to the point."
    ),
    model=critic_model,
    tool_details="A tool used to critique a statement deeply.",
    tool_params=critic_tool_params,
)

In [5]:
system_message_critic = rc.llm.SystemMessage(
    "You are a critic of the world and you provide comprehensive critiques of the world around you. You should utilize the provided tools to collect specific critiques and structure them before completing your answer."
)
Critic = rc.library.tool_call_llm(
    connected_nodes={HarshCritic, PositiveCritic, DeeperMeaningCritic},
    pretty_name="Critic",
    system_message=system_message_critic,
    model=rc.llm.OpenAILLM("gpt-4o"),
    output_type="MessageHistory",
)

In [6]:
template = (
    "I am writing a short story and I would like to analyze my introduction.\n"
    "\n"
    "Once upon a time there was a little boy who lived in a small village. He was a very kind and generous, but lacked an understanding"
    " of the world around him. He was always looking for ways to help others and make the world a better place. One day, he stumbled upon a"
    " magical book that would change his life forever. The book was filled with stories of adventure and mystery, and the promise of a better"
    " tomorrow."
)
with rc.Runner(
    executor_config=rc.run.ExecutorConfig(logging_setting="QUIET", timeout=50)
) as runner:
    response = await runner.run(
        Critic, message_history=rc.llm.MessageHistory([rc.llm.UserMessage(template)])
    )

In [7]:
response.answer

[system: You are a critic of the world and you provide comprehensive critiques of the world around you. You should utilize the provided tools to collect specific critiques and structure them before completing your answer.,
 user: I am writing a short story and I would like to analyze my introduction.
 
 Once upon a time there was a little boy who lived in a small village. He was a very kind and generous, but lacked an understanding of the world around him. He was always looking for ways to help others and make the world a better place. One day, he stumbled upon a magical book that would change his life forever. The book was filled with stories of adventure and mystery, and the promise of a better tomorrow.,
 assistant: [ToolCall(identifier='call_ug91i0CBmLkHuRLnc6AqRrBq', name='Positive_Critic', arguments={'analysis_detail': 'Once upon a time there was a little boy who lived in a small village. He was a very kind and generous, but lacked an understanding of the world around him. He was

Sometimes there may be a need to inject parameters into the subserviant tools at instance "runtime". This can be done by modifying the node creation method in the parent top level node


In [8]:
class GradeLevelHarshCritic(HarshCritic):
    def __init__(self, message_history: rc.llm.MessageHistory, grade_level: int):
        super().__init__(message_history=message_history)
        self.grade_level = grade_level

    @classmethod
    def prepare_tool(cls, tool_parameters):
        message_hist = rc.llm.MessageHistory(
            [rc.llm.UserMessage(tool_parameters["analysis_detail"])]
        )
        return cls(message_hist, tool_parameters["grade_level"])

    def system_message(self):
        return (
            super().system_message()
            + " You should provide a critique at a grade level of "
            + str(self.grade_level)
        )

In [9]:
class InjectGradeLevel(Critic):
    def __init__(self, message_history: rc.llm.MessageHistory, grade_level: int):
        super().__init__(message_history=message_history)
        self.grade_level = grade_level

    def create_node(self, tool_name: str, arguments: Dict[str, Any]) -> rc.Node:
        if tool_name == "Harsh_Critic":
            arguments["grade_level"] = self.grade_level
            return GradeLevelHarshCritic.prepare_tool(arguments)
        # note the super method will not add any parameters and work just as it always does.
        return super().create_node(tool_name, arguments)

    def connected_nodes(self) -> Set[Type[rc.Node]]:
        # note that you can add the new node to the connected nodes set
        return {GradeLevelHarshCritic, PositiveCritic, DeeperMeaningCritic}

In [10]:
template = (
    "I am writing a short story and I would like to analyze my introduction.\n"
    "\n"
    "Green Apples are a strong fruit that can be used in many different ways. They are a great source of vitamins and minerals, and can be eaten or even thrown at others you don't like"
)
# with rc.Runner(executor_config=rc.run.ExecutorConfig(logging_setting="QUIET", timeout=50)) as runner:
#     response = await runner.run(InjectGradeLevel,
#         message_history=rc.llm.MessageHistory([rc.llm.UserMessage(template)]),
#         grade_level=12)

rc.call(
    InjectGradeLevel,
    message_history=rc.llm.MessageHistory([rc.llm.UserMessage(template)]),
    grade_level=12,
)


print(response.answer)

system: You are a critic of the world and you provide comprehensive critiques of the world around you. You should utilize the provided tools to collect specific critiques and structure them before completing your answer.
user: I am writing a short story and I would like to analyze my introduction.

Green Apples are a strong fruit that can be used in many different ways. They are a great source of vitamins and minerals, and can be eaten or even thrown at others you don't like
assistant: [ToolCall(identifier='call_Eedck5TRjQ04UQUGEMAC88GW', name='Harsh_Critic', arguments={'analysis_detail': "Green Apples are a strong fruit that can be used in many different ways. They are a great source of vitamins and minerals, and can be eaten or even thrown at others you don't like", 'grade_level': 12})]
tool: This statement is a bizarre mix of nutritional advice and juvenile aggression. While it's true that green apples are nutritious, suggesting they be used as projectiles is absurd and irresponsibl